## BLIP

### Visual Question Answering
### Image-Text retrieval ( Image-Text matching)
### Image Captioning

In [0]:
%pip install torch

In [0]:
%restart_python

In [0]:
from PIL import Image
from transformers import AutoProcessor, BlipForQuestionAnswering, VisionEncoderDecoderModel, AutoTokenizer
from datasets import load_dataset
import torch

In [0]:
import requests

In [0]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"

In [0]:
image = Image.open(requests.get(url, stream=True).raw)

In [0]:
text = "What is my problem?"

In [0]:
processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base")

In [0]:
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

In [0]:
inuts = processor(images=image, text=text, return_tensors="pt")

In [0]:
output = model.generate(**inuts)

In [0]:
print(processor.decode(output[0], skip_special_tokens=True))

In [0]:
model = VisionEncoderDecoderModel.from_pretrained("facebook/vit-gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
dataset = load_dataset("coco_captions")

# 数据预处理
def preprocess_data(example):
    # 图像处理 + 文本tokenization
    example["pixel_values"] = process_image(example["image"])
    example["input_ids"] = tokenizer(example["caption"], return_tensors="pt").input_ids
    return example

dataset = dataset.map(preprocess_data)

# 模型训练
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=3
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"]
)
trainer.train()
